# 交易复盘分析

In [ ]:
from pathlib import Path
import pandas as pd

OUTPUT_PATH = Path('../data_output/output.xlsx')
SUMMARY_KEY = 'output'
SOURCE_KEY = '交易量价数据信息'

if not OUTPUT_PATH.exists():
    raise FileNotFoundError(f'找不到合并后的输出文件: {OUTPUT_PATH}')

data_out = pd.read_excel(OUTPUT_PATH, sheet_name=None)
sheet_order = list(data_out.keys())

if SUMMARY_KEY not in data_out:
    if SOURCE_KEY not in data_out:
        raise KeyError('在工作簿中找不到 `交易量价数据信息` 表，无法创建合并数据。')
    data_out[SUMMARY_KEY] = data_out[SOURCE_KEY].copy()
    sheet_order.append(SUMMARY_KEY)

print(f'已读取 {len(data_out)} 张表，目标: {SUMMARY_KEY}')


## 筛选条件

In [ ]:
summary_df = data_out[SUMMARY_KEY]
info_df = data_out['基础信息']

# 验证必需列
required_columns = ['日前中标出力', '省内中长期上网电量', '日前出清节点价格', '省内中长期均价', '日期', '公司名称', '日内实际出力']
missing_columns = [col for col in required_columns if col not in summary_df.columns]
if missing_columns:
    raise ValueError(f'{SUMMARY_KEY} 缺少以下列: {missing_columns}')

if '机组容量' not in info_df.columns or '机组名称' not in info_df.columns:
    raise ValueError(f'基础信息 缺少必需列: 机组容量 或 机组名称')

# 日期筛选配置
start_date = pd.to_datetime('2026-01-27')
end_date = pd.to_datetime('2026-01-27')

# 先转换日期列，避免重复转换
summary_df = summary_df.copy()
summary_df['日期'] = pd.to_datetime(summary_df['日期'])

# 筛选：日期范围 + 价格范围
summary_df = summary_df[
    (summary_df['日期'] >= start_date) &
    (summary_df['日期'] <= end_date) &
    (summary_df['日前出清节点价格'] >= 0) &
    (summary_df['日前出清节点价格'] <= 200)
]

print(f'筛选后剩余数据: {len(summary_df)} 行')

## 计算深调收益

In [ ]:
# 配置系数
COEFFICIENT = 660  # 深调套利折算系数（基于机组容量的标准化系数）
MID_LONG_TERM_COEFF = 160.84  # 中长期持仓均价基准
HOURS_PER_RECORD = 4  # 每条记录代表的时间间隔（96点制转24小时）

# 数值列转换
numeric_cols = ['日前中标出力', '省内中长期上网电量', '省内中长期均价', '日前出清节点价格', '日内实际出力']
has_inter = '省间中长期上网电量' in summary_df.columns and '省间中长期均价' in summary_df.columns

if has_inter:
    numeric_cols.extend(['省间中长期上网电量', '省间中长期均价'])

for col in numeric_cols:
    if col in summary_df.columns:
        summary_df[col] = pd.to_numeric(summary_df[col], errors='coerce')

# 创建匹配键并合并容量信息
summary_df['匹配键'] = summary_df['公司名称'] + summary_df['机组名称']
capacity_mapping = pd.DataFrame({
    '匹配键': info_df['公司名称'] + info_df['机组名称'],
    '机组容量': pd.to_numeric(info_df['机组容量'], errors='coerce')
})
summary_df = summary_df.merge(capacity_mapping, on='匹配键', how='left')

# 计算中长期持仓电量和持仓均价
if has_inter:
    summary_df['省间中长期上网电量'] = summary_df['省间中长期上网电量'].fillna(0)
    summary_df['省间中长期均价'] = summary_df['省间中长期均价'].fillna(0)
    contract_power = summary_df['省内中长期上网电量'] + summary_df['省间中长期上网电量']
    # 提取中长期持仓金额，避免重复计算
    intra_value = summary_df['省内中长期上网电量'] * summary_df['省内中长期均价']
    inter_value = summary_df['省间中长期上网电量'] * summary_df['省间中长期均价']
    total_contract_value = intra_value + inter_value
else:
    contract_power = summary_df['省内中长期上网电量']
    intra_value = summary_df['省内中长期上网电量'] * summary_df['省内中长期均价']
    total_contract_value = intra_value

# 计算加权平均价格（处理除零风险）
contract_price = (total_contract_value / contract_power).fillna(0).replace([float('inf'), -float('inf')], 0)

# 计算深调套利收入
condition = (summary_df['日前中标出力'] < contract_power * HOURS_PER_RECORD) & \
            (summary_df['日前出清节点价格'] < contract_price)
summary_df['深调套利收入'] = (
    (contract_power * HOURS_PER_RECORD - summary_df['日前中标出力']) *
    COEFFICIENT / summary_df['机组容量'] *
    (contract_price - summary_df['日前出清节点价格']) / HOURS_PER_RECORD
).where(condition, 0).fillna(0)

# 计算中长期平均持仓（复用 total_contract_value）
summary_df['中长期平均持仓'] = (
    total_contract_value * HOURS_PER_RECORD / MID_LONG_TERM_COEFF / summary_df['机组容量']
).fillna(0)

# 计算深调平均负荷
summary_df['单台深调平均负荷'] = summary_df['日内实际出力'] / summary_df['机组容量']

# 按机组汇总
unit_result_df = summary_df.groupby('匹配键', as_index=False).agg(
    公司名称=('公司名称', 'first'),
    日前低价时长_小时_=('日前出清节点价格', 'count'),
    现货价格_=('日前出清节点价格', 'mean'),
    深调平均负荷_=('单台深调平均负荷', 'mean'),
    中长期平均持仓_=('中长期平均持仓', 'mean'),
    深调套利_元_=('深调套利收入', 'sum')
)

# 按公司汇总
result_df = unit_result_df.groupby('公司名称', as_index=False).agg(
    日前低价时长_小时_=('日前低价时长_小时_', 'mean'),
    现货价格_=('现货价格_', 'mean'),
    深调平均负荷_=('深调平均负荷_', 'mean'),
    中长期平均持仓_=('中长期平均持仓_', 'mean'),
    深调套利_元_=('深调套利_元_', 'sum')
)

# 96点转24小时
result_df['日前低价时长_小时_'] /= HOURS_PER_RECORD

# 重命名列
result_df.columns = ['单位', '日前低价时长（小时）', '现货价格', '深调平均负荷', '中长期平均持仓', '深调套利（元）']
result_df

In [ ]:
# 高价区间统计分析
# 计算各机组在价格区间[300,1500]内的各项指标

HOURS_PER_RECORD = 4  # 每条记录代表的时间间隔（96点制转24小时）

# 重新读取原始数据（避免被前面的筛选影响）
summary_df_high = data_out[SUMMARY_KEY].copy()
info_df_high = data_out['基础信息'].copy()

# 转换日期列
summary_df_high['日期'] = pd.to_datetime(summary_df_high['日期'])

# 日期筛选配置
start_date = pd.to_datetime('2026-01-26')
end_date = pd.to_datetime('2026-01-26')

# 筛选日期范围
summary_df_high = summary_df_high[
    (summary_df_high['日期'] >= start_date) &
    (summary_df_high['日期'] <= end_date)
]

# 数值列转换
numeric_cols = ['日前中标出力', '省内中长期上网电量', '日前出清节点价格', '日内实际出力', '日内出清节点价格']
has_inter = '省间中长期上网电量' in summary_df_high.columns and '省间中长期均价' in summary_df_high.columns

if has_inter:
    numeric_cols.extend(['省间中长期上网电量'])

for col in numeric_cols:
    if col in summary_df_high.columns:
        summary_df_high[col] = pd.to_numeric(summary_df_high[col], errors='coerce')

# 创建匹配键并合并容量信息
summary_df_high['匹配键'] = summary_df_high['公司名称'] + summary_df_high['机组名称']
capacity_mapping = pd.DataFrame({
    '匹配键': info_df_high['公司名称'] + info_df_high['机组名称'],
    '机组容量': pd.to_numeric(info_df_high['机组容量'], errors='coerce')
})
summary_df_high = summary_df_high.merge(capacity_mapping, on='匹配键', how='left')

# 定义高价区间掩码
day_ahead_high_mask = (summary_df_high['日前出清节点价格'] >= 300) & (summary_df_high['日前出清节点价格'] <= 1500)
real_time_high_mask = (summary_df_high['日内出清节点价格'] >= 300) & (summary_df_high['日内出清节点价格'] <= 1500)

# 按机组汇总计算各项指标
unit_stats = []

for unit_key in summary_df_high['匹配键'].unique():
    unit_data = summary_df_high[summary_df_high['匹配键'] == unit_key]
    
    # 日前高价数据
    day_ahead_high_data = unit_data[day_ahead_high_mask[unit_data.index]]
    # 实时高价数据
    real_time_high_data = unit_data[real_time_high_mask[unit_data.index]]
    
    # 1. 日前高价时长（小时）
    day_ahead_high_hours = len(day_ahead_high_data) / HOURS_PER_RECORD
    
    # 2. 实时高价时长（小时）
    real_time_high_hours = len(real_time_high_data) / HOURS_PER_RECORD
    
    # 3. 日前现货高价均价
    day_ahead_high_avg_price = day_ahead_high_data['日前出清节点价格'].mean() if len(day_ahead_high_data) > 0 else 0
    
    # 4. 实时现货高价均价
    real_time_high_avg_price = real_time_high_data['日内出清节点价格'].mean() if len(real_time_high_data) > 0 else 0
    
    # 5. 中长期平均持仓（日前高价区间内）
    if has_inter:
        contract_power_da = day_ahead_high_data['省内中长期上网电量'].fillna(0) + day_ahead_high_data['省间中长期上网电量'].fillna(0)
    else:
        contract_power_da = day_ahead_high_data['省内中长期上网电量'].fillna(0)
    
    avg_contract_power_da = contract_power_da.mean() if len(day_ahead_high_data) > 0 else 0
    capacity = unit_data['机组容量'].iloc[0]
    mid_long_position = (avg_contract_power_da * HOURS_PER_RECORD / 2 / capacity) if capacity > 0 else 0
    
    # 6. 高价日前平均中标负荷
    day_ahead_avg_output = day_ahead_high_data['日前中标出力'].mean() / capacity if len(day_ahead_high_data) > 0 and capacity > 0 else 0
    
    # 7. 高价实时平均负荷
    real_time_avg_output = real_time_high_data['日内实际出力'].mean() / capacity if len(real_time_high_data) > 0 and capacity > 0 else 0
    
    unit_stats.append({
        '匹配键': unit_key,
        '公司名称': unit_data['公司名称'].iloc[0],
        '日前高价时长': day_ahead_high_hours,
        '实时高价时长': real_time_high_hours,
        '日前现货高价均价': day_ahead_high_avg_price,
        '实时现货高价均价': real_time_high_avg_price,
        '中长期平均持仓': mid_long_position,
        '高价日前平均中标负荷': day_ahead_avg_output,
        '高价实时平均负荷': real_time_avg_output
    })

# 创建结果DataFrame
result_df = pd.DataFrame(unit_stats)

# 按公司名称分组汇总（如果同一公司有多个机组，取均值）
final_df = result_df.groupby('公司名称', as_index=False).agg({
    '日前高价时长': 'mean',
    '实时高价时长': 'mean',
    '日前现货高价均价': 'mean',
    '实时现货高价均价': 'mean',
    '中长期平均持仓': 'mean',
    '高价日前平均中标负荷': 'mean',
    '高价实时平均负荷': 'mean'
})

# 格式化百分比列
final_df['中长期平均持仓'] = final_df['中长期平均持仓'].apply(lambda x: f"{x:.2%}")
final_df['高价日前平均中标负荷'] = final_df['高价日前平均中标负荷'].apply(lambda x: f"{x:.2%}")
final_df['高价实时平均负荷'] = final_df['高价实时平均负荷'].apply(lambda x: f"{x:.2%}")

# 重命名列
final_df.columns = ['单位', '日前高价时长', '实时高价时长', '日前现货高价均价', '实时现货高价均价', 
                    '中长期平均持仓', '高价日前平均中标负荷', '高价实时平均负荷']

final_df